<a href="https://colab.research.google.com/github/sumanthks-01/DL-project/blob/main/Disease_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Classify plant diseases using greyscale images from the datasets.

In [12]:
!git clone --depth 1 https://github.com/spMohanty/PlantVillage-Dataset.git
!mv PlantVillage-Dataset/raw ./plantvillage_raw

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163214, done.
remote: Counting objects: 100% (163214/163214), done.
remote: Compressing objects: 100% (163119/163119), done.
remote: Total 163214 (delta 93), reused 163212 (delta 93), pack-reused 0 (from 0)
Receiving objects: 100% (163214/163214), 2.00 GiB | 40.29 MiB/s, done.
Resolving deltas: 100% (93/93), done.
Updating files: 100% (182401/182401), done.


In [14]:
import tensorflow as tf

data_dir = './plantvillage_raw/grayscale'

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    color_mode='grayscale',
    batch_size=128,
    validation_split=0.2,
    subset='training',
    seed=123)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    color_mode='grayscale',
    batch_size=128,
    validation_split=0.2,
    subset='validation',
    seed=123)

class_names = train_ds.class_names
print(class_names)

Found 54305 files belonging to 38 classes.
Using 43444 files for training.
Found 54305 files belonging to 38 classes.
Using 10861 files for validation.
['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_bl

In [15]:
def normalize_img(image, label):
  """Normalizes images by scaling pixel values to the range [0, 1]."""
  return tf.cast(image, tf.float32) / 255.0, label

train_ds = train_ds.map(normalize_img)
val_ds = val_ds.map(normalize_img)

train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 38)             │         4,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,308,966 (12.62 MB)

 Trainable params: 3,308,966 (12.62 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 53s 135ms/step - accuracy: 0.3911 - loss: 2.3074 - val_accuracy: 0.7163 - val_loss: 0.9749
Epoch 2/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 20s 59ms/step - accuracy: 0.7454 - loss: 0.8408 - val_accuracy: 0.8068 - val_loss: 0.6357
Epoch 3/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 20s 59ms/step - accuracy: 0.8424 - loss: 0.5099 - val_accuracy: 0.8200 - val_loss: 0.5782
Epoch 4/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 20s 59ms/step - accuracy: 0.8866 - loss: 0.3591 - val_accuracy: 0.8277 - val_loss: 0.5606
Epoch 5/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 20s 58ms/step - accuracy: 0.9184 - loss: 0.2586 - val_accuracy: 0.8181 - val_loss: 0.6437
Epoch 6/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 20s 59ms/step - accuracy: 0.9375 - loss: 0.1942 - val_accuracy: 0.8372 - val_loss: 0.6055
Epoch 7/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 20s 58ms/step - accuracy: 0.9503 - loss: 0.1501 - val_accuracy: 0.8449 - val_loss: 0.5994
Epoch 8/10
340/340 ━━━━━━━━━━━━━━━━━━━━ 20s 58ms/step - accuracy: 0.9623 - loss: 0.1159 -

In [27]:
import numpy as np
from PIL import Image

def predict_plant_disease(image_path, model, class_names):
    image_size = (256, 256) # Adjust to match your model's input size
    img = Image.open(image_path).convert('L') # Convert to grayscale
    img = img.resize(image_size) # Resize to match model input size
    img_array = np.array(img) / 255.0 # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0) # Add batch dimension
    img_array = np.expand_dims(img_array, axis=-1) # Add channel dimension for grayscale

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    predicted_class_name = class_names[predicted_class_index]

    if 'healthy' in predicted_class_name:
        return f"The plant is healthy."
    else:
        return f"The plant is diseased. The predicted disease is: {predicted_class_name}"

# Example usage (you'll need to replace with an actual image path)
result = predict_plant_disease('/content/plantvillage_raw/raw/grayscale/Soybean___healthy/005f6aee-12a7-4d83-9e9a-027c7cc34dee___RS_HL 6536.JPG', model, class_names)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
The plant is healthy.


In [18]:
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8620 - loss: 0.5933
Validation Loss: 0.6120132207870483
Validation Accuracy: 0.8583924174308777
